# 2025 DL Lab8: RL Assignment_Super Mario World

**Your Answer:**    
Hi I'm XXX, XXXXXXXXXX.

## Overview
This project implements a **Deep Reinforcement Learning** pipeline to train an autonomous agent for Super Mario World. Leveraging the **Proximal Policy Optimization (PPO)** algorithm, the system interacts with the **stable-retro** environment to master the YoshiIsland1 level. Key components include a custom Vision Backbone for extracting features from raw pixel data and a suite of Environment Wrappers that handle frame preprocessing, action discretization, and reward shaping to facilitate efficient learning.

Reward function implement  
should do something in the beginning (monster attack)  
Custom PPO implement  
pre train weight 差不多，主要是 reward function  
model weight capacity 1GB  
class name 不要動 (可以新增，但是原本有的不要動)

## Imports

In [1]:
import os
import numpy as np
import retro
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.vec_env import VecNormalize

from eval import evaluate_policy, record_video
from custom_policy import VisionBackbonePolicy, CustomPPO

/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

In [2]:
# Game Settings
GAME = "SuperMarioWorld-Snes"
STATE = "YoshiIsland1"

# Training Settings
# TOTAL_STEPS = 0x1400000 # 20,971,520
# TOTAL_STEPS = 0x0A00000 # 10,485,760
TOTAL_STEPS = 0X3200000 # 52,428,800
TRAIN_CHUNK = 0x0040000 #    262,144
N_ENVS = 16

# Evaluation & Recording Settings
EVAL_EPISODES = 3
EVAL_MAX_STEPS = 18000
RECORD_STEPS = 1800

# Directories
LOG_DIR = "./runs_smw"
VIDEO_DIR       = os.path.join(LOG_DIR, "videos")
CKPT_DIR        = os.path.join(LOG_DIR, "checkpoints")
TENSORBOARD_LOG = os.path.join(LOG_DIR, "tb")

os.makedirs(LOG_DIR,   exist_ok=True)
os.makedirs(CKPT_DIR,  exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

## Environment Functions

In [3]:
from wrappers import make_base_env
def _make_env_thunk(game: str, state: str):
    """Return a function that creates an environment (for multiprocessing)."""
    def _thunk():
        return make_base_env(game, state)
    return _thunk

def make_vec_env(game: str, state: str, n_envs: int, use_subproc: bool = True):
    """Create a vectorized environment (multiple envs running in parallel)."""
    env_fns = [_make_env_thunk(game, state) for _ in range(n_envs)]
    
    if use_subproc and n_envs > 1:
        vec_env = SubprocVecEnv(env_fns)
    else:
        vec_env = DummyVecEnv(env_fns)

    return vec_env

## Initialize Env & Model

In [4]:
# 1. Create Training Environment
train_env = make_vec_env(GAME, STATE, n_envs=N_ENVS)
# train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10., clip_reward=10.)
print(f"Environment created: {GAME} - {STATE} with {N_ENVS} parallel envs.")

checkpoint_path = "None" # 6225920 (19) 有破壞
checkpoint_path = "runs_smw/preserved/Enc5_67.zip"

best_mean = -1e18
trained = 0
round_idx = 0

# 2. Initialize Model
if os.path.exists(checkpoint_path):
    # 讀取現有模型
    model = CustomPPO.load(
        checkpoint_path, 
        env=train_env,
        device="cuda:0" # 確保使用 GPU
    )
    trained = model.num_timesteps
    round_idx = int(trained / TRAIN_CHUNK)
    print(f"[Sucess] Loaded model from {checkpoint_path}")
    print(f"trained: {trained}, round_index: {round_idx}")
else:
    print(f"[Fail] Can't load {checkpoint_path}. Will use new model")
    model = CustomPPO(
        VisionBackbonePolicy,
        train_env,
        policy_kwargs   = dict(normalize_images=False),
        n_epochs        = 4,
        n_steps         = 512,
        batch_size      = 512,
        learning_rate   = 1e-4,
        verbose         = 1,
        gamma           = 0.96875,
        kl_coef         = 1,
        clip_range      = 0.125,
        ent_coef        = 0.0375,
        tensorboard_log = TENSORBOARD_LOG,
    )

Environment created: SuperMarioWorld-Snes - YoshiIsland1 with 16 parallel envs.
[Sucess] Loaded model from runs_smw/preserved/Enc5_67.zip
trained: 17825792, round_index: 68


In [5]:
# model.save("policy")

In [6]:
# import os
# import glob
# from custom_policy import CustomPPO
# from eval import record_video  # 確保 eval.py 在同一目錄下

# # ================= 設定區 =================
# target_numbers = [59, 63, 67]

# # ================= 執行迴圈 =================
# print(f"準備測試以下 Checkpoints: {target_numbers}")

# for num in target_numbers:
#     model_path = os.path.join(CKPT_DIR, f"Enc5_{num}.zip")
    
#     # 檢查檔案是否存在
#     if not os.path.exists(model_path):
#         print(f"⚠️ 找不到檔案: {model_path}，跳過。")
#         continue
    
#     print(f"\n[{num}] 正在載入模型: {model_path} ...")
    
#     try:
#         # 1. 載入模型 (不需要 env 參數也能載入權重)
#         # 如果你有改過 CustomPPO 的參數，load 會自動讀取 zip 裡的設定
#         model = CustomPPO.load(model_path, device="auto") # device="auto" 會自動用 GPU
        
#         # 2. 錄製影片
#         prefix_name = f"test_{num}"
#         print(f"[{num}] 正在錄影 (長度 {RECORD_STEPS} steps)...")
        
#         record_video(
#             model=model,
#             game=GAME,
#             state=STATE,
#             out_dir=VIDEO_DIR,
#             video_len=RECORD_STEPS,
#             prefix=prefix_name
#         )
#         print(f"✅ 完成！影片已儲存為 {prefix_name}.mp4")
        
#     except Exception as e:
#         print(f"❌ 發生錯誤 (Model: {num}): {e}")

# print("\n所有測試結束。")

## Training Loop

In [ ]:
try:
    while trained < TOTAL_STEPS:
        round_idx += 1
        chunk = min(TRAIN_CHUNK, TOTAL_STEPS - trained)
        # chunk = 2000
        label = "Enc5"
        tagged_label = f"{label}_{int(trained/TRAIN_CHUNK)}"

        print(f"\n=== Round {round_idx} | Learn {chunk} steps (Total trained: {trained}) ===")
        
        # --- Train ---
        model.learn(total_timesteps=chunk, reset_num_timesteps=False, tb_log_name=label)
        trained += chunk

        # --- Save Checkpoint ---
        ckpt_path = os.path.join(CKPT_DIR, f"{tagged_label}.zip")
        model.save(ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

        # --- Evaluate ---
        mean_ret, best_ret = evaluate_policy(
            model,
            GAME,
            STATE,
            n_episodes=EVAL_EPISODES,
            max_steps=EVAL_MAX_STEPS,
        )
        print(f"[EVAL] Mean Return: {mean_ret:.3f}, Best Return: {best_ret:.3f}")

        # --- Save Best Model ---
        # if mean_ret > best_mean:
        #     best_mean = mean_ret
        #     best_path = os.path.join(LOG_DIR, "best_model.zip")
        #     model.save(best_path)
        #     print(f"New best record. Saved to {best_path}")

        # --- Record Video ---
        out_path = os.path.join(VIDEO_DIR, label)
        os.makedirs(out_path,  exist_ok=True)
        record_video(
            model,
            GAME,
            STATE,
            VIDEO_DIR,
            video_len=RECORD_STEPS,
            prefix=f"{label}/{tagged_label}_{mean_ret:.2f}",
        )

except KeyboardInterrupt:
    print("\nTraining interrupted manually.")

finally:
    train_env.close()
    print("Training finished. Environment closed.")
    
"""
tensorboard --logdir=./runs_smw/tb
"""


=== Round 69 | Learn 262144 steps (Total trained: 17825792) ===
Logging to ./runs_smw/tb/Enc5_0


/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

---------------------------------
| time/              |          |
|    fps             | 1074     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 17833984 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 894         |
|    iterations         | 2           |
|    time_elapsed       | 18          |
|    total_timesteps    | 17842176    |
| train/                |             |
|    approx_kl          | 0.013940858 |
|    entropy_loss       | -1.62       |
|    explained_variance | 0.927       |
|    learning_rate      | 0.0001      |
|    loss               | 0.367       |
|    mean_step_reward   | 0.28193492  |
|    n_updates          | 8708        |
|    policyGradLoss     | 0.00121     |
|    value_loss         | 1.4         |
---------------------------------------
---------------------------------------
| time/                 |             |
| 

Saved checkpoint: ./runs_smw/checkpoints/Enc5_68.zip
[EVAL] Mean Return: 520.960, Best Return: 525.627
Saved video to ./runs_smw/videos/Enc5/Enc5_68_520.96.mp4

=== Round 70 | Learn 262144 steps (Total trained: 18087936) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1106     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 18096128 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 894         |
|    iterations         | 2           |
|    time_elapsed       | 18          |
|    total_timesteps    | 18104320    |
| train/                |             |
|    approx_kl          | 0.011413626 |
|    entropy_loss       | -1.7        |
|    explained_variance | 0.951       |
|    learning_rate      | 0.0001      |
|    loss               | 1.09        |
|    mean_step_reward   |

Saved checkpoint: ./runs_smw/checkpoints/Enc5_69.zip
[EVAL] Mean Return: 524.401, Best Return: 530.401
Saved video to ./runs_smw/videos/Enc5/Enc5_69_524.40.mp4

=== Round 71 | Learn 262144 steps (Total trained: 18350080) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1131     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 18358272 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 921         |
|    iterations         | 2           |
|    time_elapsed       | 17          |
|    total_timesteps    | 18366464    |
| train/                |             |
|    approx_kl          | 0.009198898 |
|    entropy_loss       | -1.66       |
|    explained_variance | 0.965       |
|    learning_rate      | 0.0001      |
|    loss               | 0.832       |
|    mean_step_reward   |

...

Saved checkpoint: ./runs_smw/checkpoints/Enc5_175.zip
[EVAL] Mean Return: 542.932, Best Return: 549.599
Saved video to ./runs_smw/videos/Enc5/Enc5_175_542.93.mp4

=== Round 177 | Learn 262144 steps (Total trained: 46137344) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1128     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 46145536 |
---------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 931        |
|    iterations         | 2          |
|    time_elapsed       | 17         |
|    total_timesteps    | 46153728   |
| train/                |            |
|    approx_kl          | 0.01783223 |
|    entropy_loss       | -1.57      |
|    explained_variance | 0.99       |
|    learning_rate      | 0.0001     |
|    loss               | 0.326      |
|    mean_step_reward   | 0.456368

Saved checkpoint: ./runs_smw/checkpoints/Enc5_176.zip
[EVAL] Mean Return: 543.462, Best Return: 550.128
Saved video to ./runs_smw/videos/Enc5/Enc5_176_543.46.mp4

=== Round 178 | Learn 262144 steps (Total trained: 46399488) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1081     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 46407680 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 896         |
|    iterations         | 2           |
|    time_elapsed       | 18          |
|    total_timesteps    | 46415872    |
| train/                |             |
|    approx_kl          | 0.017213155 |
|    entropy_loss       | -1.67       |
|    explained_variance | 0.984       |
|    learning_rate      | 0.0001      |
|    loss               | 0.532       |
|    mean_step_reward 

Saved checkpoint: ./runs_smw/checkpoints/Enc5_177.zip
[EVAL] Mean Return: 540.701, Best Return: 547.368
Saved video to ./runs_smw/videos/Enc5/Enc5_177_540.70.mp4

=== Round 179 | Learn 262144 steps (Total trained: 46661632) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1110     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 46669824 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 899         |
|    iterations         | 2           |
|    time_elapsed       | 18          |
|    total_timesteps    | 46678016    |
| train/                |             |
|    approx_kl          | 0.018571056 |
|    entropy_loss       | -1.65       |
|    explained_variance | 0.987       |
|    learning_rate      | 0.0001      |
|    loss               | 0.877       |
|    mean_step_reward 

Saved checkpoint: ./runs_smw/checkpoints/Enc5_178.zip
[EVAL] Mean Return: 543.260, Best Return: 549.927
Saved video to ./runs_smw/videos/Enc5/Enc5_178_543.26.mp4

=== Round 180 | Learn 262144 steps (Total trained: 46923776) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1179     |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 46931968 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 939         |
|    iterations         | 2           |
|    time_elapsed       | 17          |
|    total_timesteps    | 46940160    |
| train/                |             |
|    approx_kl          | 0.014438041 |
|    entropy_loss       | -1.58       |
|    explained_variance | 0.987       |
|    learning_rate      | 0.0001      |
|    loss               | 0.652       |
|    mean_step_reward 

Saved checkpoint: ./runs_smw/checkpoints/Enc5_179.zip
[EVAL] Mean Return: 543.359, Best Return: 550.025
Saved video to ./runs_smw/videos/Enc5/Enc5_179_543.36.mp4

=== Round 181 | Learn 262144 steps (Total trained: 47185920) ===
Logging to ./runs_smw/tb/Enc5_0
---------------------------------
| time/              |          |
|    fps             | 1154     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 47194112 |
---------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 934       |
|    iterations         | 2         |
|    time_elapsed       | 17        |
|    total_timesteps    | 47202304  |
| train/                |           |
|    approx_kl          | 0.0122112 |
|    entropy_loss       | -1.64     |
|    explained_variance | 0.967     |
|    learning_rate      | 0.0001    |
|    loss               | 0.306     |
|    mean_step_reward   | 0.3493291 |
|    n_u

## Display Video

In [ ]:
from IPython.display import Video
import glob
# label = "Dec22A"

# list_of_files = glob.glob(os.path.join(VIDEO_DIR, label, '*.mp4')) 
# if list_of_files:
#     latest_file = max(list_of_files, key=os.path.getctime)
#     print(f"Playing: {latest_file}")
#     latest_file = "runs_smw/videos/Dec22A/Dec22A_73_596.54.mp4"
#     print(f"Playing: {latest_file}")
#     display(Video(latest_file, embed=True, width=768))
# else:
#     print("No videos found yet.")
    
video = "./runs_smw/videos/Dec22A/Dec22A_73_596.54.mp4"
# display(Video(video, embed=True, width=768))

In [ ]:
import cv2

cap = cv2.VideoCapture("runs_smw/videos/test_16.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Frame-by-Frame", frame)

    # 關鍵：這裡等待按鍵。按 'n' 鍵跳到下一幀，按 'q' 離開
    key = cv2.waitKey(0) 
    if key == ord('q'):
        break
    elif key == ord('n'):
        continue

cap.release()
cv2.destroyAllWindows()